In [1]:
import pandas as pd
import modulo as md
import requests
import time 
from bs4 import BeautifulSoup
import numpy as np


In [2]:
headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'}

Estos son los equipos ingleses que estan en primera y segunda division en la temporada el año actual 2023-2024 sin embargo a los equipos que hayan estado en 2 divisiones en los ultimos años se tomaran como equipos de la menor division en la que hayan estado por ejemplo el Luton equipo recien ascendido a premier league aunque actualmente este en primera division en general será tratado como equipo de segunda division.

In [5]:
ingleses=pd.read_csv('dataset\All_countries_teams\England_teams.csv')
españoles=pd.read_csv('dataset\All_countries_teams\Spain_teams.csv')
italianos=pd.read_csv('dataset\All_countries_teams\Italy_teams.csv')
franceses=pd.read_csv('dataset\All_countries_teams\France_teams.csv')
alemanes=pd.read_csv('dataset\All_countries_teams\Germany_teams.csv')

In [3]:
def descenso(nombre, codigoE,id_ml,temporada, resultados,equipos_descendidos):
    print('El ' + nombre+' estuvo en segunda division en la temporada '+temporada + ' al momento van '+str(len(resultados))+ ' equipos', end='\n\r')
    equipo_descendido={}
    equipo_descendido['Nombre']=nombre
    equipo_descendido['id Web']=codigoE
    equipo_descendido['id_ml']=id_ml
    equipos_descendidos.append(equipo_descendido)
    #activo descendio
    descendio=True
    return descendio





def get_top_teams_mean(teams,years:int):
    league_level=1
    
    #league_level: la division de los equipos a los que les tomaras las medias
    #teams: el dataframe donde se encuentran los equipos
    #anios: la cantidad de años que debe estar el equipo en esa division para ser considerado parte de ella
    
    #creo una lista donde voy a guardar 1 dataframe por equipo con el rendimiento de ese equipo durante las ultimas 5 temporadas
 
    resultados=[]

    '''defino una variable que me dice de en que division  deberian estar todos los equipos durante las temporadas,
    a los equipos que esten en segunda division o hayan estado en segunda durante los ultimos 5 años solo les estudiare 3 años de su historial
    y si durante esos 3 años estuvo en tercera division solo se le valeran los años que estuvo en primera o segunda.'''
    

    temporadas=md.listnEditions(5,md.year())
    

    equipos_descendidos=[]
    #recorro el dataframe que guarda todos los equipos 
    for index, row in teams.iterrows():
        #en el principio asumo que el equipo no ha descendido en las ultimas 5 temporadas
        descendio=False
        # elijo el equipo
        nombre=row['Nombre']
        codigoE=row['id Web']
        id_ml=row['id_ml']
        anios_en_nivel=0
        
        # creo una lista donde voy guardar los resulados del equipo en cada temporada
        temporadas_equipo=[]

        #recorro las temporadas obteniendo los datos del equipo esa temporada
        for temporada in temporadas:
            #muestro el equipo y la temporada que estoy scrapeando
            print(nombre,temporada, end='\r')
            url=f'https://fbref.com/es/equipos/{codigoE}/{temporada}/all_comps/'
            res = requests.get(url, headers=headers)
            soup=BeautifulSoup(res.text, 'html.parser')

            #si el equipo en algun momento estuvo de otra division diferente a la 1 creo un diccionario con sus datos y lo agrego a una lista
            # de diccionarios de equipos descendidos
            nivelactual=md.league_level(soup)
            
            if not nivelactual:

                descendio=descenso(nombre, codigoE,id_ml,temporada, resultados,equipos_descendidos)

                break

            nivelactual=int(nivelactual)
            
            if nivelactual > league_level and anios_en_nivel < years:

                
                descendio=descenso(nombre, codigoE,id_ml,temporada, resultados,equipos_descendidos)
                
                break 

            #obtengo los resultados para el equipo en la temporada
            equipo=md.get_players_mean(res,url)
            #añado una columna llamada temporada donde guardare de que temporada es el registro
            equipo['temporada']=temporada
            #añado los resultados a la lista
            temporadas_equipo.append(equipo)
            anios_en_nivel+=1


        #si el equipo no descendio 
        if not descendio:
            
            #apilo todos los registros del equipo en un solo dataframe 
            resultado = pd.concat(temporadas_equipo, ignore_index=True)
            resultados.append(resultado)

    equipos = pd.concat(resultados, ignore_index=True)
    if nivelactual:
        equipos_descendidos=md.concat_dicts(equipos_descendidos)
    
    return [equipos, equipos_descendidos]
    
        



In [5]:
equipos_ingleses=get_top_teams_mean(ingleses,3)

El Bournemouth estuvo en segunda en la temporada 2021-2022 al momento van 2 equipos
El Brentford estuvo en segunda en la temporada 2020-2021 al momento van 2 equipos
El Burnley estuvo en segunda en la temporada 2022-2023 al momento van 3 equipos
El Fulham estuvo en segunda en la temporada 2021-2022 al momento van 6 equipos
El Luton Town estuvo en segunda en la temporada 2022-2023 al momento van 7 equipos
El Nott'ham Forest estuvo en segunda en la temporada 2021-2022 al momento van 10 equipos
El Sheffield Utd estuvo en segunda en la temporada 2022-2023 al momento van 10 equipos
El Birmingham City estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El Blackburn estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El Bristol City estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El Cardiff City estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El Coventry City estuvo en segunda en la temporada 2022-2023 al momento 

In [6]:
equipos_ingleses[0].to_csv('dataset/Equipos_primera/England.csv',index=False)
equipos_ingleses[1].to_csv('dataset/Equipos_segunda/England.csv',index=False)

In [7]:
equipos_alemanes=get_top_teams_mean(alemanes,3)

El Bochum estuvo en segunda en la temporada 2020-2021 al momento van 2 equipos
El Darmstadt 98 estuvo en segunda en la temporada 2022-2023 al momento van 2 equipos
El Heidenheim estuvo en segunda en la temporada 2022-2023 al momento van 5 equipos
El Werder Bremen estuvo en segunda en la temporada 2021-2022 al momento van 13 equipos
El Braunschweig estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Düsseldorf estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Elversberg estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Greuther Fürth estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Hamburger SV estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Hannover 96 estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Hansa Rostock estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Holstein Kiel estuvo en segunda en la temporada 2022-2023 al mo

In [8]:
equipos_alemanes[0].to_csv('dataset/Equipos_primera/Germany.csv',index=False)
equipos_alemanes[1].to_csv('dataset/Equipos_segunda/Germany.csv',index=False)

In [9]:
equipos_franceses=get_top_teams_mean(franceses,3)

El Clermont Foot estuvo en segunda en la temporada 2020-2021 al momento van 1 equipos
El Le Havre estuvo en segunda en la temporada 2022-2023 al momento van 1 equipos
El Metz estuvo en segunda en la temporada 2022-2023 al momento van 6 equipos
El Toulouse estuvo en segunda en la temporada 2021-2022 al momento van 14 equipos
El Ajaccio estuvo en segunda en la temporada 2021-2022 al momento van 14 equipos
El Amiens estuvo en segunda en la temporada 2022-2023 al momento van 14 equipos
El Annecy estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Auxerre estuvo en segunda en la temporada 2021-2022 al momento van 15 equipos
El Bastia estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Bordeaux estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Caen estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Dunkerque estuvo en segunda en la temporada 2022-2023 al momento van 15 equipos
El Grenoble estuvo en segunda

In [10]:
equipos_franceses[0].to_csv('dataset/Equipos_primera/France.csv',index=False)
equipos_franceses[1].to_csv('dataset/Equipos_segunda/France.csv',index=False)

In [11]:
equipos_españoles=get_top_teams_mean(españoles,3)

El Alavés estuvo en segunda en la temporada 2022-2023 al momento van 0 equipos
El Almería estuvo en segunda en la temporada 2021-2022 al momento van 0 equipos
El Girona estuvo en segunda en la temporada 2021-2022 al momento van 7 equipos
El Granada estuvo en segunda en la temporada 2022-2023 al momento van 7 equipos
El Las Palmas estuvo en segunda en la temporada 2022-2023 al momento van 7 equipos
El Mallorca estuvo en segunda en la temporada 2020-2021 al momento van 7 equipos
El Rayo Vallecano estuvo en segunda en la temporada 2020-2021 al momento van 8 equipos
El Albacete estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El Alcorcón estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El Burgos estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El Cartagena estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El CD Eldense estuvo en segunda en la temporada 2022-2023 al momento van 13 equipos
El CD Mirandés estuv

In [12]:
equipos_españoles[0].to_csv('dataset/Equipos_primera/Spain.csv',index=False)
equipos_españoles[1].to_csv('dataset/Equipos_segunda/Spain.csv',index=False)

In [8]:
equipos_italianos=get_top_teams_mean(italianos,3)

El Cagliari estuvo en segunda division en la temporada 2022-2023 al momento van 2 equipos
El Empoli estuvo en segunda division en la temporada 2020-2021 al momento van 2 equipos
El Frosinone estuvo en segunda division en la temporada 2022-2023 al momento van 3 equipos
El Genoa estuvo en segunda division en la temporada 2022-2023 al momento van 3 equipos
El Lecce estuvo en segunda division en la temporada 2021-2022 al momento van 7 equipos
El Monza estuvo en segunda division en la temporada 2021-2022 al momento van 8 equipos
El Salernitana estuvo en segunda division en la temporada 2020-2021 al momento van 10 equipos
El Ascoli estuvo en segunda division en la temporada 2022-2023 al momento van 13 equipos
El Bari estuvo en segunda division en la temporada 2022-2023 al momento van 13 equipos
El Brescia estuvo en segunda division en la temporada 2022-2023 al momento van 13 equipos
El Calcio Lecco 1912 estuvo en segunda division en la temporada 2022-2023 al momento van 13 equipos
El Catanza

In [10]:
equipos_italianos[0].to_csv('dataset/Equipos_primera/Italy.csv',index=False)
equipos_italianos[1].to_csv('dataset/Equipos_segunda/Italy.csv',index=False)

In [4]:
def get_second_teams_mean(teams):
    league_level=2
    
    #league_level: la division de los equipos a los que les tomaras las medias
    #teams: el dataframe donde se encuentran los equipos
    #anios: la cantidad de años que debe estar el equipo en esa division para ser considerado parte de ella
    
    #creo una lista donde voy a guardar 1 dataframe por equipo con el rendimiento de ese equipo durante las ultimas 5 temporadas
 
    resultados=[]

    '''defino una variable que me dice de en que division  deberian estar todos los equipos durante las temporadas,
    a los equipos que esten en segunda division o hayan estado en segunda durante los ultimos 5 años solo les estudiare 3 años de su historial
    y si durante esos 3 años estuvo en tercera division solo se le valeran los años que estuvo en primera o segunda.'''
    

    temporadas=md.listnEditions(3,md.year())
    
    #recorro el dataframe que guarda todos los equipos 
    for index, row in teams.iterrows():
        #en el principio asumo que el equipo no ha descendido en las ultimas 5 temporadas
        
        # elijo el equipo
        nombre=row['Nombre']
        codigoE=row['id Web']
       
        # creo una lista donde voy guardar los resulados del equipo en cada temporada
        temporadas_equipo=[]

        #recorro las temporadas obteniendo los datos del equipo esa temporada
        for temporada in temporadas:
            #muestro el equipo y la temporada que estoy scrapeando
            print(nombre,temporada, end='\r')
            url=f'https://fbref.com/es/equipos/{codigoE}/{temporada}/all_comps/'
            res = requests.get(url, headers=headers)
            soup=BeautifulSoup(res.text, 'html.parser')

            #si el equipo en algun momento estuvo de otra division diferente a la 1 creo un diccionario con sus datos y lo agrego a una lista
            # de diccionarios de equipos descendidos
            nivelactual=md.league_level(soup)
            
            if not nivelactual:
                print(f'la temporada {temporada} del {nombre} no cuenta para el registro')
                break

            nivelactual=int(nivelactual)
            
            if nivelactual > league_level:
                print(f'la temporada {temporada} del {nombre} no cuenta para el registro')
                break 
            
            
            #obtengo los resultados para el equipo en la temporada
            equipo=md.get_players_mean(res,url)
            if  equipo.empty:
                print(f'la temporada {temporada} del {nombre} no cuenta para el registro')
                break
            #añado una columna llamada temporada donde guardare de que temporada es el registro
            equipo['temporada']=temporada
            #añado los resultados a la lista
            temporadas_equipo.append(equipo)
            

        if len(temporadas_equipo) > 1:
        #apilo todos los registros del equipo en un solo dataframe 
            resultado = pd.concat(temporadas_equipo, ignore_index=True)
            resultados.append(resultado)
        elif len(temporadas_equipo)==1:
            
            resultado=temporadas_equipo[0]
            resultados.append(resultado)
        
        
        print(len(resultados))
    equipos = pd.concat(resultados, ignore_index=True)
    
    
    return equipos

In [5]:
second_england=pd.read_csv('dataset\Equipos_segunda\England.csv')
second_france=pd.read_csv('dataset\Equipos_segunda\France.csv')
second_germany=pd.read_csv('dataset\Equipos_segunda\Germany.csv')
second_italy=pd.read_csv('dataset\Equipos_segunda\Italy.csv')
second_spain=pd.read_csv('dataset\Equipos_segunda\Spain.csv')

In [13]:
second_england_mean=get_second_teams_mean(second_england)

1ournemouth 2020-2021
2rentford 2020-2021
3urnley 2020-2021
4ulham 2020-2021
5uton Town 2020-2021
6ott'ham Forest 2020-2021
7heffield Utd 2020-2021
8irmingham City 2020-2021
9lackburn 2020-2021
10istol City 2020-2021
11rdiff City 2020-2021
12ventry City 2020-2021
13ddersfield 2020-2021
la temporada 2020-2021 del Hull City no cuenta para el registro
14
la temporada 2022-2023 del Ipswich Town no cuenta para el registro
14
15ddlesbrough 2020-2021
16llwall 2020-2021
17rwich City 2020-2021
la temporada 2022-2023 del Plymouth Argyle no cuenta para el registro
17
18eston 2020-2021
19R 2020-2021
la temporada 2021-2022 del Rotherham Utd no cuenta para el registro
20
la temporada 2022-2023 del Sheffield Weds no cuenta para el registro
20
21oke City 2020-2021
la temporada 2021-2022 del Sunderland no cuenta para el registro
22
23ansea City 2020-2021
24tford 2020-2021
25st Brom 2020-2021


In [15]:
second_england_mean.to_csv('dataset\Equipos_segunda\England.csv', index=False)

In [18]:
second_france_mean=get_second_teams_mean(second_france)

1lermont Foot 2020-2021
2e Havre 2020-2021
3etz 2020-2021
4oulouse 2020-2021
5jaccio 2020-2021
6miens 2020-2021
la temporada 2021-2022 del Annecy no cuenta para el registro
7
8uxerre 2020-2021
la temporada 2020-2021 del Bastia no cuenta para el registro
9
10rdeaux 2020-2021
11en 2020-2021
la temporada 2022-2023 del Dunkerque no cuenta para el registro
11
12enoble 2020-2021
13ingamp 2020-2021
14ris FC 2020-2021
15u FC 2020-2021
la temporada 2020-2021 del Quevilly-Rouen no cuenta para el registro
16
17dez Aveyron 2020-2021
18int-Étienne 2020-2021
la temporada 2021-2022 del Stade Laval no cuenta para el registro
19
20oyes 2020-2021
la temporada 2022-2023 del US Concarneau no cuenta para el registro
20
21lenciennes 2020-2021


In [ ]:
second_france_mean.to_csv('dataset\Equipos_segunda\England.csv', index=False)


In [8]:
second_germany_mean=get_second_teams_mean(second_germany)

1ochum 2020-2021
2armstadt 98 2020-2021
3eidenheim 2020-2021
4erder Bremen 2020-2021
la temporada 2021-2022 del Braunschweig no cuenta para el registro
5
6üsseldorf 2020-2021
la temporada 2022-2023 del Elversberg no cuenta para el registro
6
7reuther Fürth 2020-2021
8amburger SV 2020-2021
9annover 96 2020-2021
la temporada 2020-2021 del Hansa Rostock no cuenta para el registro
10
11lstein Kiel 2020-2021
la temporada 2021-2022 del Kaiserslautern no cuenta para el registro
12
13rlsruher 2020-2021
la temporada 2021-2022 del Magdeburg no cuenta para el registro
14
15rnberg 2020-2021
la temporada 2022-2023 del Osnabrück no cuenta para el registro
15
16derborn 07 2020-2021
17halke 04 2020-2021
18. Pauli 2020-2021
la temporada 2022-2023 del Wehen no cuenta para el registro
18


In [9]:
second_germany_mean.to_csv('dataset\Equipos_segunda\Germany.csv', index=False)

In [8]:
second_italy_mean=get_second_teams_mean(second_italy)

1agliari 2020-2021
2mpoli 2020-2021
3rosinone 2020-2021
4enoa 2020-2021
5ecce 2020-2021
6onza 2020-2021
7alernitana 2020-2021
8scoli 2020-2021
la temporada 2021-2022 del Bari no cuenta para el registro
9
10escia 2020-2021
la temporada 2022-2023 del Calcio Lecco 1912 no cuenta para el registro
10
la temporada 2022-2023 del Catanzaro no cuenta para el registro
10
11ttadella 2020-2021
la temporada 2020-2021 del Como no cuenta para el registro
12
13senza 2020-2021
14emonese 2020-2021
la temporada 2022-2023 del FeralpiSalò no cuenta para el registro
14
la temporada 2021-2022 del Modena no cuenta para el registro
15
la temporada 2021-2022 del Palermo no cuenta para el registro
16
17rma 2020-2021
18sa 2020-2021
la temporada 2022-2023 del Reggiana no cuenta para el registro
18
la temporada 2021-2022 del Südtirol no cuenta para el registro
19
la temporada 2020-2021 del Ternana no cuenta para el registro
20
21nezia 2020-2021


In [9]:
second_italy_mean.to_csv('dataset\Equipos_segunda\Italy.csv', index=False)

In [6]:
second_spain_mean=get_second_teams_mean(second_spain)

1lavés 2020-2021
2lmería 2020-2021
3irona 2020-2021
4ranada 2020-2021
5as Palmas 2020-2021
6allorca 2020-2021
7ayo Vallecano 2020-2021
la temporada 2021-2022 del Albacete no cuenta para el registro
8
la temporada 2022-2023 del Alcorcón no cuenta para el registro
8
la temporada 2020-2021 del Burgos no cuenta para el registro
9
10rtagena 2020-2021
la temporada 2022-2023 del CD Eldense no cuenta para el registro
10
11 Mirandés 2020-2021
12bar 2020-2021
13panyol 2020-2021
la temporada 2021-2022 del FC Andorra no cuenta para el registro
14
15esca 2020-2021
16ganés 2020-2021
17vante 2020-2021
18iedo 2020-2021
la temporada 2022-2023 del Racing Ferrol no cuenta para el registro
18
la temporada 2021-2022 del Racing Sant no cuenta para el registro
19
la temporada 2022-2023 del SD Amorebieta no cuenta para el registro
19
20orting Gijón 2020-2021
21nerife 2020-2021
22lladolid 2020-2021
la temporada 2022-2023 del Villarreal B no cuenta para el registro
22
23ragoza 2020-2021


In [7]:
second_spain_mean.to_csv('dataset\Equipos_segunda\Spain.csv', index=False)
